в случае отсутствия установить библиотеку \
pip install pandas

In [37]:
# импорт необходимых библиотек
import datetime
import pandas as pd
import numpy as np

В данных для задачи "Предсказание социально-демографических характеристик пользователя" в колонке **event_timestamp** указано московское время (GMT +3). \
Используя данные из колонки **region** приводится время к региональному

In [2]:
# загрузка данных с историей просмотров пользователей
all_events = pd.read_csv("data/all_events.csv")
all_events.head()

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 13:08:30+03:00,Tatarstan Republic,smartphone,browser,Android,Chrome Mobile,60,video_395879,10813370
1,2024-06-01 14:30:00+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,60,video_216518,10512324
2,2024-06-01 18:48:12+03:00,Novosibirsk Oblast,smartphone,mobile app,Android,Rutube,121,video_41225,10951137
3,2024-06-01 16:32:36+03:00,Moscow,desktop,browser,Windows,Yandex Browser,2324,video_215886,10912434
4,2024-06-01 00:57:04+03:00,Moscow Oblast,smartphone,mobile app,Android,Rutube,6830,video_43631,10223585


In [3]:
# загрузка данных с историей просмотров пользователей с известными целевыми переменными
train_events = pd.read_csv("data/train_events.csv")
train_events.head()

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,512,video_362960,10245341
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,5647,video_96775,10894333
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,71,video_116245,10452976


In [4]:
# размерность данных
all_events.shape, train_events.shape

((8439624, 9), (1759616, 9))

In [5]:
# проверка на возможность объединения данных
all_events.columns == train_events.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [30]:
# объединение данных
events_df = pd.concat([all_events, train_events])
events_df.head()

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 13:08:30+03:00,Tatarstan Republic,smartphone,browser,Android,Chrome Mobile,60,video_395879,10813370
1,2024-06-01 14:30:00+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,60,video_216518,10512324
2,2024-06-01 18:48:12+03:00,Novosibirsk Oblast,smartphone,mobile app,Android,Rutube,121,video_41225,10951137
3,2024-06-01 16:32:36+03:00,Moscow,desktop,browser,Windows,Yandex Browser,2324,video_215886,10912434
4,2024-06-01 00:57:04+03:00,Moscow Oblast,smartphone,mobile app,Android,Rutube,6830,video_43631,10223585


Описание признаков
* event_date : Дата события
* viewer_uid : Идентификатор пользователя
* region : Регион пользователя
* rutube_video_id : Идентификатор видео
* ua_device_type : Устройство пользователя
* ua_client_type : Приложение/браузер 
* ua_os : Операционная система устройства пользователя
* ua_client_name : Веб-браузер/приложение, с которого пользователь просматривал видео
* total_watchtime : Время просмотра в секундах

In [31]:
# проверка соответствия размерности данных
len(all_events) + len(train_events) == len(events_df)

True

In [32]:
# очистка памяти
del all_events
del train_events

In [6]:
# загрузка данных с временными зонами регионов
region_tz = pd.read_csv("data/region_tz.csv")
region_tz.head()

,Регион en,Регион ru,timezone,lat,lon
0,Kursk Oblast,Курск,UTC+3,51.7388,36.1874
1,Ulyanovsk,Ульяновск,UTC+4,54.3520,48.3663
2,Volgograd Oblast,Волгоград,UTC+4,48.7080,44.5130
3,Novgorod Oblast,Великий Новгород,UTC+3,58.5161,31.2756
4,Tambov Oblast,Тамбов,UTC+3,52.7311,41.4423


In [34]:
# размерность данных
events_df.shape, region_tz.shape

((10199240, 9), (116, 5))

In [7]:
# объединение данных с историей просмотров и временными зонами регионов
events_df = train_events.merge(region_tz, left_on="region", right_on="Регион en")
events_df.head() 

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,Регион en,Регион ru,timezone,lat,lon
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243,Chelyabinsk,Челябинск,UTC+5,55.1644,61.4368
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,512,video_362960,10245341,Bashkortostan Republic,Уфа,UTC+5,54.7388,55.9721
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,5647,video_96775,10894333,St.-Petersburg,Санкт-Петербург,UTC+3,59.9343,30.3351
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092,Moscow,Москва,UTC+3,55.7558,37.6173
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,71,video_116245,10452976,Moscow,Москва,UTC+3,55.7558,37.6173


In [8]:
# размерность данных
events_df.shape

(1759616, 14)

In [9]:
# очистка памяти
del region_tz

In [10]:
# признаки данных
events_df.columns

Index(['event_timestamp', 'region', 'ua_device_type', 'ua_client_type',
       'ua_os', 'ua_client_name', 'total_watchtime', 'rutube_video_id',
       'viewer_uid', 'Регион en', 'Регион ru', 'timezone', 'lat', 'lon'],
      dtype='object')

In [11]:
# соответствие типов данных
events_df.dtypes

event_timestamp     object
region              object
ua_device_type      object
ua_client_type      object
ua_os               object
ua_client_name      object
total_watchtime      int64
rutube_video_id     object
viewer_uid           int64
Регион en           object
Регион ru           object
timezone            object
lat                float64
lon                float64
dtype: object

Тип данных - дата события **нерелевантен**

In [12]:
# приведение в соответствие типа данных даты события
events_df["event_timestamp"] = pd.to_datetime(events_df["event_timestamp"])

In [13]:
# соответствие типов данных
events_df.dtypes

event_timestamp    datetime64[ns, UTC+03:00]
region                                object
ua_device_type                        object
ua_client_type                        object
ua_os                                 object
ua_client_name                        object
total_watchtime                        int64
rutube_video_id                       object
viewer_uid                             int64
Регион en                             object
Регион ru                             object
timezone                              object
lat                                  float64
lon                                  float64
dtype: object

In [14]:
# проверка корректности объединения данных 
events_df.shape

(1759616, 14)

Данные объединены **без** потерь

In [16]:
def get_utc_tz(tz: str) -> int:
    """ получение часовой метки временной зоны

    Args:
        tz (str): временная зона

    Returns:
        int: часовая метка
    """

    return int(tz[3:])

In [17]:
# получение временной метки
events_df["timezone"] = events_df["timezone"].map(get_utc_tz)

In [18]:
#  разница во времени относительно GMT +3
events_df["gmt diff"] = events_df["timezone"] - 3

In [19]:
# вывод данных
events_df.head()

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,Регион en,Регион ru,timezone,lat,lon,gmt diff
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243,Chelyabinsk,Челябинск,5,55.1644,61.4368,2
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,512,video_362960,10245341,Bashkortostan Republic,Уфа,5,54.7388,55.9721,2
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,5647,video_96775,10894333,St.-Petersburg,Санкт-Петербург,3,59.9343,30.3351,0
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092,Moscow,Москва,3,55.7558,37.6173,0
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,71,video_116245,10452976,Moscow,Москва,3,55.7558,37.6173,0


In [20]:
# получение регионального времени
current_event_time = []
# проход в цикле по дате события и разницы во времени относительно GMT +3
for event_time, gmt_diff in events_df.iloc[:,[0, -1]].values:
    # разница во времени
    delta = datetime.timedelta(hours=gmt_diff)
    # получение регионально времени в зависимости от разницы во времени
    if gmt_diff < 0:
        current_event_time.append(event_time - delta)
    elif gmt_diff > 0:
        current_event_time.append(event_time + delta)
    else:
        current_event_time.append(event_time)

In [21]:
# признак регионального времени
events_df["current_event_time"] = current_event_time

In [22]:
# очистка памяти
del current_event_time

In [23]:
# временной диапазон
events_df["current_event_time"].agg(["min", "max"])

min   2024-06-01 00:00:02+03:00
max   2024-07-01 08:40:13+03:00
Name: current_event_time, dtype: datetime64[ns, UTC+03:00]

Предоставлены данные за месяц **июнь** дня (с 1 июня по 1 июля 2024 года)

Формирование признака времени дня 

00.00 до 06.00 **ночь** \
06.00 до 12.00 **утро** \
12.00 до 18.00 **день** \
18.00 до 00.00 **вечер**

In [24]:
# региональное время
events_df["current_time"] = events_df["current_event_time"].dt.time

In [25]:
# временные диапазоны времени суток
cut_labels = ["ночь", "утро", "день", "вечер"]
cut_bins = [datetime.time.fromisoformat("00:00:00"),
            datetime.time.fromisoformat("06:00:00"),
            datetime.time.fromisoformat("12:00:00"),
            datetime.time.fromisoformat("18:00:00"),
            datetime.time.fromisoformat("23:59:59")]

In [26]:
# временные диапазоны времени суток
events_df["time_cut"] = pd.cut(events_df["current_time"], bins=cut_bins, labels=cut_labels)

In [27]:
# количественное значение по категориям
(events_df["time_cut"].value_counts(normalize=True) * 100).round(2).astype("str") + " %"

time_cut
вечер    36.85 %
день     28.89 %
утро     19.92 %
ночь     14.34 %
Name: proportion, dtype: object

Данные по категориям времени суток **сбалансированы** и **релевантны**

In [28]:
def make_harmonic_features(value: int, angle: str="cos", period: int=24) -> float:
    """преобразование времени в координатную плоскость

    Args:
        value (int): время
        angle (str, optional): функция преобразования. Defaults to "cos".
        period (int, optional): период. Defaults to 24.

    Returns:
        float: координата на плоскости
    """
    value *= 2 *  np.pi / period
    match angle:
        case "cos":
            return np.cos(value)
        case "sin":
            return np.sin(value)

In [29]:
# получение координат для часа
events_df["current_hour_x"] = events_df["current_event_time"].dt.hour.map(make_harmonic_features)
events_df["current_hour_y"] = events_df["current_event_time"].dt.hour.map(lambda x: make_harmonic_features(x, angle="sin"))

In [30]:
# получение координат для минут
events_df["current_minute_x"] = events_df["current_event_time"].dt.minute.map(lambda x: make_harmonic_features(x, period=60))
events_df["current_minute_y"] = events_df["current_event_time"].dt.minute.map(lambda x: make_harmonic_features(x, angle="sin", period=60))

In [31]:
#  признак дня недели
events_df["current_weekday"] = events_df["current_event_time"].dt.weekday

In [32]:
def is_day_off(weekday: int) -> bool:
    """проверяет - является ли день просмотра выходным днём

    Args:
        weekday (int): день недели

    Returns:
        bool: является ли день просмотра выходным днём
    """
    return weekday in [5, 6]

In [33]:
#  признак выходного дня недели
events_df["is_day_off"] = events_df["current_weekday"].map(is_day_off) 

In [34]:
# получение координат для дня недели
events_df["current_weekday_x"] = events_df["current_weekday"].map(lambda x: make_harmonic_features(x, period=7))
events_df["current_weekday_y"] = events_df["current_weekday"].map(lambda x: make_harmonic_features(x, angle="sin", period=7))

In [35]:
events_df.head()

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,Регион en,...,current_time,time_cut,current_hour_x,current_hour_y,current_minute_x,current_minute_y,current_weekday,is_day_off,current_weekday_x,current_weekday_y
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243,Chelyabinsk,...,08:40:58,утро,-0.500000,0.866025,-0.500000,-8.660254e-01,5,True,-0.222521,-0.974928
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,512,video_362960,10245341,Bashkortostan Republic,...,21:33:24,вечер,0.707107,-0.707107,-0.951057,-3.090170e-01,5,True,-0.222521,-0.974928
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,5647,video_96775,10894333,St.-Petersburg,...,21:30:43,вечер,0.707107,-0.707107,-1.000000,1.224647e-16,5,True,-0.222521,-0.974928
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092,Moscow,...,23:03:42,вечер,0.965926,-0.258819,0.951057,3.090170e-01,5,True,-0.222521,-0.974928
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,71,video_116245,10452976,Moscow,...,22:48:09,вечер,0.866025,-0.500000,0.309017,-9.510565e-01,5,True,-0.222521,-0.974928


In [38]:
events_df.shape

(1759616, 26)

In [36]:
events_df.to_csv("data/data.csv", index=False)